In [2]:
import pymysql
import os 
from dotenv import load_dotenv

load_dotenv()

conn = pymysql.connect(

host=os.getenv("MYSQL_HOST"),       
user=os.getenv("MYSQL_USER_NAME"),
password=os.getenv("MYSQL_USER_PASSWORD"),
db=os.getenv("MYSQL_DATABASE"),
port=int(os.getenv("MYSQL_PORT")),

charset='utf8mb4',
cursorclass=pymysql.cursors.DictCursor
)

In [3]:
query = """
SELECT *
FROM shoes_test2;
"""

with conn.cursor() as cursor:
    cursor.execute(query)
    result = cursor.fetchall()

In [4]:
import pandas as pd 

data = pd.DataFrame(result)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2979 entries, 0 to 2978
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   idx            2979 non-null   int64         
 1   color          2979 non-null   object        
 2   sub_category   2979 non-null   object        
 3   season         2979 non-null   object        
 4   goods_name     2979 non-null   object        
 5   thumbnail_url  2979 non-null   object        
 6   is_soldout     2979 non-null   int64         
 7   goods_url      2979 non-null   object        
 8   brand          2979 non-null   object        
 9   normal_price   2979 non-null   int64         
 10  price          2979 non-null   int64         
 11  created_at     2979 non-null   datetime64[ns]
 12  updated_at     2979 non-null   datetime64[ns]
dtypes: datetime64[ns](2), int64(4), object(7)
memory usage: 302.7+ KB


In [9]:
# s3 버킷 

import boto3 
from botocore.exceptions import ClientError

service_name = os.getenv("service_name")
endpoint_url = os.getenv("endpoint_url")
region_name = os.getenv("region_name")
access_key = os.getenv("access_key")
secret_key = os.getenv("secret_key")

s3 = boto3.client(service_name, endpoint_url=endpoint_url, aws_access_key_id=access_key,
                  aws_secret_access_key=secret_key,
                  config=boto3.session.Config(signature_version="s3v4"))

In [10]:
import boto3

bucket_name = "web-assets-prod"
prefix = "shoes/"

paginator = s3.get_paginator("list_objects_v2")
page_iterator = paginator.paginate(Bucket=bucket_name, Prefix=prefix)

all_keys = []

for page in page_iterator:
    contents = page.get("Contents", [])
    for obj in contents:
        all_keys.append(obj["Key"])


In [11]:
len(all_keys)

2934

In [13]:
all_keys[:-1]

['clothes/',
 'clothes/1000241.jpg',
 'clothes/1001138.jpg',
 'clothes/1005904.jpg',
 'clothes/1006232.jpg',
 'clothes/1006697.jpg',
 'clothes/1008039.jpg',
 'clothes/1009421.jpg',
 'clothes/1009471.jpg',
 'clothes/1009804.jpg',
 'clothes/1011285.jpg',
 'clothes/1014253.jpg',
 'clothes/1014956.jpg',
 'clothes/1015136.jpg',
 'clothes/1015137.jpg',
 'clothes/1016976.jpg',
 'clothes/1017008.jpg',
 'clothes/1017136.jpg',
 'clothes/1017888.jpg',
 'clothes/1017894.jpg',
 'clothes/1020133.jpg',
 'clothes/1023788.jpg',
 'clothes/1023790.jpg',
 'clothes/1024780.jpg',
 'clothes/1025218.jpg',
 'clothes/1027192.jpg',
 'clothes/1027583.jpg',
 'clothes/1027608.jpg',
 'clothes/1029532.jpg',
 'clothes/1029538.jpg',
 'clothes/1031254.jpg',
 'clothes/1031256.jpg',
 'clothes/1031258.jpg',
 'clothes/1031259.jpg',
 'clothes/1031260.jpg',
 'clothes/1032569.jpg',
 'clothes/1032570.jpg',
 'clothes/1032581.jpg',
 'clothes/1033282.jpg',
 'clothes/1033849.jpg',
 'clothes/1033854.jpg',
 'clothes/1034454.jpg',
 'c

In [14]:
s3_keys = [obj["Key"] for obj in all_keys.get("Contents", [])]

AttributeError: 'list' object has no attribute 'get'

In [15]:
len(s3_keys[1:])

NameError: name 's3_keys' is not defined

In [14]:
s3_map = {
    key.split("/")[-1].replace(".jpg", ""): key 
    for key in all_keys[1:]
}

In [15]:
len(s3_map)

2933

In [18]:
def extract_id(url: str) -> str:
    try:
        return url.split("/")[6]
    except IndexError:
        return ""

In [19]:
s3_map

{'1000241': 'clothes/1000241.jpg',
 '1001138': 'clothes/1001138.jpg',
 '1005904': 'clothes/1005904.jpg',
 '1006232': 'clothes/1006232.jpg',
 '1006697': 'clothes/1006697.jpg',
 '1008039': 'clothes/1008039.jpg',
 '1009421': 'clothes/1009421.jpg',
 '1009471': 'clothes/1009471.jpg',
 '1009804': 'clothes/1009804.jpg',
 '1011285': 'clothes/1011285.jpg',
 '1014253': 'clothes/1014253.jpg',
 '1014956': 'clothes/1014956.jpg',
 '1015136': 'clothes/1015136.jpg',
 '1015137': 'clothes/1015137.jpg',
 '1016976': 'clothes/1016976.jpg',
 '1017008': 'clothes/1017008.jpg',
 '1017136': 'clothes/1017136.jpg',
 '1017888': 'clothes/1017888.jpg',
 '1017894': 'clothes/1017894.jpg',
 '1020133': 'clothes/1020133.jpg',
 '1023788': 'clothes/1023788.jpg',
 '1023790': 'clothes/1023790.jpg',
 '1024780': 'clothes/1024780.jpg',
 '1025218': 'clothes/1025218.jpg',
 '1027192': 'clothes/1027192.jpg',
 '1027583': 'clothes/1027583.jpg',
 '1027608': 'clothes/1027608.jpg',
 '1029532': 'clothes/1029532.jpg',
 '1029538': 'clothes

In [16]:
data["image_id"] = data["thumbnail_url"].apply(lambda x: str(x).split("/")[6].strip())
data["s3_path"] = data["image_id"].map(s3_map)


In [23]:
redata = data[data["s3_path"].isna()== True]

In [ ]:
import requests 
from PIL import Image
from io import BytesIO
from typing import List
import pandas as pd

uploaded_keys = []

for idx, row in redata.iterrows():
    try:
        image_url = row["thumbnail_url"]
        response = requests.get(image_url, timeout=10)
        image = Image.open(BytesIO(response.content)).convert("RGB")

        # 고유 키 생성
        image_id = image_url.split("/")[6]
        s3_key = f"shoes/{image_id}.jpg"
        
        buffer = BytesIO()
        image.save(buffer, format="JPEG")
        buffer.seek(0)

        s3.put_object(
            Bucket=bucket_name,
            Key=s3_key,
            Body=buffer,
            ContentType="image/jpeg"
        )

        print(f"✅ [{idx}] 업로드 성공: s3://{bucket_name}/{s3_key}")
        uploaded_keys.append(s3_key)

    except Exception as e:
        print(f"❌ [{idx}] 업로드 실패: {e}")


In [ ]:
data["s3_path"].isna()==True

0       shoes/1115031.jpg
1       shoes/3802627.jpg
2                     NaN
3                     NaN
4       shoes/1635193.jpg
              ...        
2974    shoes/1447508.jpg
2975    shoes/4999078.jpg
2976    shoes/4999211.jpg
2977    shoes/4700114.jpg
2978    shoes/3969744.jpg
Name: s3_path, Length: 2979, dtype: object

In [5]:
for key in list(s3_map.keys())[:10]:
    print(repr(key))


NameError: name 's3_map' is not defined

In [78]:
test_id = data["image_id"].iloc[0]
print("테스트 ID:", test_id)
print("s3_map에 존재?", test_id in s3_map)

테스트 ID: 4471126
s3_map에 존재? True


In [51]:
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

load_dotenv()

# SQLAlchemy 엔진 생성
engine = create_engine(
    f"mysql+pymysql://{os.getenv('MYSQL_USER_NAME')}:{os.getenv('MYSQL_USER_PASSWORD')}@{os.getenv('MYSQL_HOST')}:{os.getenv('MYSQL_PORT')}/{os.getenv('MYSQL_DATABASE')}"
)

# DataFrame을 새 테이블에 업로드
data.to_sql(
    name="mens_table_refine",  # 새 테이블명
    con=engine,
    index=False,
    if_exists="replace"      # 없으면 생성, 있으면 덮어쓰기 ("append"로 바꾸면 추가)
)

print("mens_table_refine 테이블로 업로드 완료!")


mens_table_refine 테이블로 업로드 완료!


In [94]:
def validate_match(row):
    if pd.isna(row["s3_path"]):
        return False
    return row["image_id"] in row["s3_path"]

data["match"] = data.apply(validate_match, axis=1)


In [98]:
data["match"]

0        True
1        True
2        True
3        True
4        True
         ... 
31297    True
31298    True
31299    True
31300    True
31301    True
Name: match, Length: 31302, dtype: bool

In [ ]:
data["s3_path"] = data["image_id"].apply(lambda x : "https://kr.object.iwinv.+ x + ".jpg")

SyntaxError: unterminated string literal (detected at line 1) (417286371.py, line 1)

In [46]:
data["s3_path"][1]s

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3735778384.py, line 1)

In [35]:
data.drop(["match"],axis=1, inplace=True)

In [45]:
data[["image_id", "s3_path"]]s

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3873713200.py, line 1)

In [12]:
data

,idx,color,sub_category,season,goods_name,thumbnail_url,is_soldout,goods_url,brand,normal_price,price,created_at,updated_at,image_id
0,1,화이트,캔버스/단화,"봄, 여름, 가을, 겨울",단테 - 화이트 / 22127100,https://image.msscdn.net/images/goods_img/2019...,0,https://www.musinsa.com/products/1115031,닥터마틴,170000,101990,2025-04-21 17:57:13,2025-04-21 17:57:13,1115031
1,2,화이트,캔버스/단화,"봄, 여름, 가을, 겨울",Steve Shoes / WHPGE1101A,https://image.msscdn.net/images/goods_img/2024...,0,https://www.musinsa.com/products/3802627,후아유,29900,14950,2025-04-21 17:57:13,2025-04-21 17:57:13,3802627
2,3,화이트,캔버스/단화,"봄, 여름, 가을, 겨울",토너먼트 레더 스니커즈(1TM01734F109),https://image.msscdn.net/images/goods_img/2021...,0,https://www.musinsa.com/products/2233099,케즈,79000,55300,2025-04-21 17:57:13,2025-04-21 17:57:13,2233099
3,4,화이트,캔버스/단화,"봄, 여름, 가을, 겨울",2402 뮬 FUR white 화이트,https://image.msscdn.net/images/goods_img/2019...,0,https://www.musinsa.com/products/1090658,수페르가,85000,17000,2025-04-21 17:57:13,2025-04-21 17:57:13,1090658
4,5,화이트,캔버스/단화,"봄, 여름, 가을, 겨울",레더 코트 스니커즈 [화이트],https://image.msscdn.net/images/goods_img/2020...,0,https://www.musinsa.com/products/1635193,무신사 스탠다드,69900,55890,2025-04-21 17:57:13,2025-04-21 17:57:13,1635193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2974,2975,브라운,스포츠/캐주얼 샌들,여름,EKAN-MEPPEN(메펜)(BROWN),https://image.msscdn.net/images/goods_img/2020...,0,https://www.musinsa.com/products/1447508,에칸,49000,49000,2025-04-21 17:57:26,2025-04-21 17:57:26,1447508
2975,2976,브라운,스포츠/캐주얼 샌들,여름,Kingdom 킹덤 남성 피셔맨 샌들_3color 5cm,https://image.msscdn.net/images/goods_img/2025...,0,https://www.musinsa.com/products/4999078,마크모크,110000,93500,2025-04-21 17:57:26,2025-04-21 17:57:26,4999078
2976,2977,브라운,스포츠/캐주얼 샌들,여름,New Super 뉴 슈퍼 남녀공용 베스트 피셔맨 샌들_3color 1.5cm,https://image.msscdn.net/images/goods_img/2025...,0,https://www.musinsa.com/products/4999211,마크모크,92000,78200,2025-04-21 17:57:26,2025-04-21 17:57:26,4999211
2977,2978,브라운,스포츠/캐주얼 샌들,여름,(MEN) 코르크 미드솔 스웨이드 크로그 샌들 브라운 (MA39K1SC1D),https://image.msscdn.net/images/goods_img/2025...,0,https://www.musinsa.com/products/4700114,숄,186000,186000,2025-04-21 17:57:26,2025-04-21 17:57:26,4700114


In [105]:
import pandas as pd

query = "SELECT * FROM shoes_refine;"
df = pd.read_sql(query, con=engine)

In [7]:
data

,idx,color,sub_category,season,goods_name,thumbnail_url,is_soldout,goods_url,brand,normal_price,price,created_at,updated_at,image_id
0,1,화이트,캔버스/단화,"봄, 여름, 가을, 겨울",단테 - 화이트 / 22127100,https://image.msscdn.net/images/goods_img/2019...,0,https://www.musinsa.com/products/1115031,닥터마틴,170000,101990,2025-04-21 17:57:13,2025-04-21 17:57:13,1115031
1,2,화이트,캔버스/단화,"봄, 여름, 가을, 겨울",Steve Shoes / WHPGE1101A,https://image.msscdn.net/images/goods_img/2024...,0,https://www.musinsa.com/products/3802627,후아유,29900,14950,2025-04-21 17:57:13,2025-04-21 17:57:13,3802627
2,3,화이트,캔버스/단화,"봄, 여름, 가을, 겨울",토너먼트 레더 스니커즈(1TM01734F109),https://image.msscdn.net/images/goods_img/2021...,0,https://www.musinsa.com/products/2233099,케즈,79000,55300,2025-04-21 17:57:13,2025-04-21 17:57:13,2233099
3,4,화이트,캔버스/단화,"봄, 여름, 가을, 겨울",2402 뮬 FUR white 화이트,https://image.msscdn.net/images/goods_img/2019...,0,https://www.musinsa.com/products/1090658,수페르가,85000,17000,2025-04-21 17:57:13,2025-04-21 17:57:13,1090658
4,5,화이트,캔버스/단화,"봄, 여름, 가을, 겨울",레더 코트 스니커즈 [화이트],https://image.msscdn.net/images/goods_img/2020...,0,https://www.musinsa.com/products/1635193,무신사 스탠다드,69900,55890,2025-04-21 17:57:13,2025-04-21 17:57:13,1635193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2974,2975,브라운,스포츠/캐주얼 샌들,여름,EKAN-MEPPEN(메펜)(BROWN),https://image.msscdn.net/images/goods_img/2020...,0,https://www.musinsa.com/products/1447508,에칸,49000,49000,2025-04-21 17:57:26,2025-04-21 17:57:26,1447508
2975,2976,브라운,스포츠/캐주얼 샌들,여름,Kingdom 킹덤 남성 피셔맨 샌들_3color 5cm,https://image.msscdn.net/images/goods_img/2025...,0,https://www.musinsa.com/products/4999078,마크모크,110000,93500,2025-04-21 17:57:26,2025-04-21 17:57:26,4999078
2976,2977,브라운,스포츠/캐주얼 샌들,여름,New Super 뉴 슈퍼 남녀공용 베스트 피셔맨 샌들_3color 1.5cm,https://image.msscdn.net/images/goods_img/2025...,0,https://www.musinsa.com/products/4999211,마크모크,92000,78200,2025-04-21 17:57:26,2025-04-21 17:57:26,4999211
2977,2978,브라운,스포츠/캐주얼 샌들,여름,(MEN) 코르크 미드솔 스웨이드 크로그 샌들 브라운 (MA39K1SC1D),https://image.msscdn.net/images/goods_img/2025...,0,https://www.musinsa.com/products/4700114,숄,186000,186000,2025-04-21 17:57:26,2025-04-21 17:57:26,4700114


In [ ]:
df["s3_path"] = df["s3_path"].apply(lambda x : "https://kr.object.iwinv.kr/" + x)

In [2]:
df

NameError: name 'df' is not defined

In [6]:
data["image_id"] = data["goods_url"].apply(lambda x : x.split("/")[-1])

In [63]:
data["s3_path"][1]

'https://kr.object.iwinv.kr/web-assets-prod/shoes/1354129.jpg'

In [1]:
data["image_id"][1]

NameError: name 'data' is not defined

In [65]:
data.to_sql(
    name="shoes_refine",  # 기존 테이블 덮어쓰기 or 새로 저장
    con=engine,
    index=False,
    if_exists="replace"   # "replace": 기존 테이블 삭제 후 저장 / "append": 데이터 추가
)


: 